In [15]:
%load_ext autoreload

In [20]:
%autoreload 2

In [1]:
from src.parsers import mimic, hirid
from src.modeling import discovery, plots, querier
from src.utils import constants

In [2]:
import sys
# import win32com.client
import os
import pandas as pd

def setup_io_config(root_path):
    """
    Input - Output config. Add dataset paths
    :root_path -> Repo path which contains 'data' and 'res' folders
    """

    # MIMIC
    is_shortcut = True if "data.lnk" in os.listdir(root_path) else False 
    
    if (is_shortcut):
        path_shortcut =  os.path.join(root_path, "data.lnk")
        shell = win32com.client.Dispatch("WScript.Shell")
        mimic_data = shell.CreateShortCut(path_shortcut).Targetpath
    else:
        mimic_data = os.path.join(f"{root_path}", "data") 
    mimic_path = os.path.join(f"{root_path}", "results")

    # HIRID
    hirid_data = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1/raw_stage/'
    hirid_path = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1'
    
    return mimic_data, mimic_path, hirid_data, hirid_path

def setup_stratification_config():
    gender="MF"
    age_b=40
    age_a=80 
    ethnicity="WHITE" 
    lab_mapping= constants.LAB_MAPPING
    b_w = [(0,6), (6,12)]
    a_w = [(0,4), (4,8), (8,12)]
    before_windows = b_w
    after_windows = a_w
    return gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows

In [3]:
# IO Config
# root_path ="C:\\Users\\danco\\My Drive\\Master\\Datasets\\MIMIC iii"
root_path = "/Users/pavan/Library/CloudStorage/GoogleDrive-f20190038@hyderabad.bits-pilani.ac.in/My Drive/TAU/Code/DrugLab"
data, res, raw_path, res_path = setup_io_config(root_path=root_path)

# Stratification Config
gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows = setup_stratification_config()

In [ ]:
# MIMIC
mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, load="AUTOMATIC_MAPPING_MIMIC")
m_med1, m_med2, m_labs = mimic_parser.parse(use_pairs=False, load_from_raw=False, load_raw_chartevents=False)

## Querier
mimic_data_querier = querier.DatasetQuerier(
    data = data,
    res = res,
    t_labs=m_labs, 
    t_med1=m_med1, 
    t_med2=m_med2,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
# query pairs for all medication and lab tests
m_final_lab_med_data = mimic_data_querier.generate_med_lab_data(before_windows, after_windows)
# Querying pairs for a single medication and lab test
b_w = [(0,6), (6,12)]
a_w = [(0,4), (4,8), (8,12)]
med_lab_pair_1 = mimic_data_querier.query('Insulin - Regular', 'Glucose', b_w, a_w)

## Discovery Analysis for the queried medication and lab test pairs in the chosen before and after windows
analyzer = discovery.ClinicalDiscoveryAnalysis(m_final_lab_med_data)
pvals_med_lab = analyzer.analyze(before_windows, after_windows)
sig_med_lab = analyzer.generate_significant(pvals_med_lab.dropna(subset=["TTest Paired"]))

## Plots
plotter = plots.ClinicalPlotAnalysis(
    data = data,
    res = res,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
m_corrs_data_df = plotter.plot(m_final_lab_med_data, m_labs, before_windows=before_windows, after_windows=after_windows)

In [5]:
# HIRID
hirid_mapping = constants.HIRID_MAPPING
lab_parts = (20,30)
hirid_parser = hirid.HiRiDParser(data=raw_path, res=res_path, gender=gender, age_b=age_b, age_a=age_a, load="MANUAL_MAPPING_HIRID")
h_med1, h_med2, h_labs = hirid_parser.parse(lab_parts=lab_parts)
hirid_data_querier = querier.DatasetQuerier(
    data = raw_path,
    res = res_path,
    t_labs=h_labs, 
    t_med1=h_med1, 
    t_med2=h_med2,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
)
final_h_final_lab_med_data, raw_h_final_lab_med_data = hirid_data_querier.generate_med_lab_data(before_windows, after_windows, lab_parts=lab_parts)
## Discovery Analysis for the queried medication and lab test pairs in the chosen before and after windows
analyzer = discovery.ClinicalDiscoveryAnalysis(final_h_final_lab_med_data)
types_l = ["abs", "mean", "std", "trends"]
pvals_med_lab = analyzer.analyze(before_windows, after_windows, min_patients=100, types_l=types_l)

pval, hard, bonferroni, fdr = analyzer.generate_significant(pvals_med_lab.dropna(subset=["TTest Paired"]))
fdr.to_csv(f"temp_hirid_sig_med_lab_pairs_fdr_{lab_parts}.csv")
bonferroni.to_csv(f"temp_hirid_sig_med_lab_pairs_bonferroni_{lab_parts}.csv")
hard.to_csv(f"temp_hirid_sig_med_lab_pairs_hard_{lab_parts}.csv")
pval.to_csv(f"temp_hirid_sig_med_lab_pairs_pval_{lab_parts}.csv")

/Users/pavan/Desktop/TAU/DrugLab/src/parsers/hirid.py:85: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  labs = pd.read_csv(path)
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/hirid.py:85: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  labs = pd.read_csv(path)
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/hirid.py:85: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  labs = pd.read_csv(path)
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/hirid.py:85: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  labs = pd.read_csv(path)
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/hirid.py:85: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  labs = pd.read_csv(path)
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/hirid.py:85: DtypeWarning: Col

### Part (10,20)

In [ ]:
fdr

,Lab Name,Med Name,Before Window (in Hours),After Window (in Hours),Mannwhitneyu Test,TTest Independent,TTest Paired,No of Patients,Type,BonferroniPvals,FDR Benjamini Corrected
28,Calcium.ionized [Moles/volume] in Blood,Benuron supp 1000 mg,"(0, 6)","(0, 4)",1.153321e-06,0.000005,1.902946e-15,157,abs,3.824921e-13,2.490544e-13
30,Glucose [Moles/volume] in Serum or Plasma,Benuron supp 1000 mg,"(0, 6)","(0, 4)",1.026727e-04,0.000394,8.537959e-08,191,abs,1.716130e-05,4.195735e-06
36,Potassium [Moles/volume] in Blood,Benuron supp 1000 mg,"(0, 6)","(0, 4)",2.431379e-03,0.001420,1.043715e-07,159,abs,2.097868e-05,4.195735e-06
37,Respiratory rate,Benuron supp 1000 mg,"(0, 6)","(0, 4)",1.101796e-08,0.003749,7.012093e-06,270,abs,1.409431e-03,1.566034e-04
38,Sodium [Moles/volume] in Blood,Benuron supp 1000 mg,"(0, 6)","(0, 4)",9.826408e-06,0.000122,2.478154e-15,157,abs,4.981089e-13,2.490544e-13
39,Calcium.ionized [Moles/volume] in Blood,Dafalgan Brausetbl 1g,"(6, 12)","(0, 4)",1.642939e-02,0.021772,8.501055e-06,121,abs,1.708712e-03,1.708712e-04
74,Metronidazole tabl 200 mg,Dormicum inj 1 mg/ml,"(0, 6)","(0, 4)",2.309590e-03,0.003959,1.909392e-06,229,abs,3.837877e-04,4.818827e-05
77,Heart rate,Ebrantil Inj Lsg,"(0, 6)","(0, 4)",2.919198e-01,0.323706,4.668884e-04,190,abs,9.384457e-02,7.218813e-03
91,Metronidazole tabl 200 mg,Fentanyl inj 50 µg/ml,"(0, 6)","(0, 4)",4.586420e-02,0.048784,2.076345e-04,729,abs,4.173453e-02,3.477878e-03
189,Respiratory rate,Tracrium inj 10 mg/ml,"(0, 6)","(0, 4)",3.797414e-06,0.002019,6.965143e-05,149,abs,1.399994e-02,1.272722e-03


In [7]:
bonferroni

,Lab Name,Med Name,Before Window (in Hours),After Window (in Hours),Mannwhitneyu Test,TTest Independent,TTest Paired,No of Patients,Type,BonferroniPvals,FDR Benjamini Corrected
28,Calcium.ionized [Moles/volume] in Blood,Benuron supp 1000 mg,"(0, 6)","(0, 4)",1.153321e-06,0.000005,1.902946e-15,157,abs,3.824921e-13,2.490544e-13
30,Glucose [Moles/volume] in Serum or Plasma,Benuron supp 1000 mg,"(0, 6)","(0, 4)",1.026727e-04,0.000394,8.537959e-08,191,abs,1.716130e-05,4.195735e-06
36,Potassium [Moles/volume] in Blood,Benuron supp 1000 mg,"(0, 6)","(0, 4)",2.431379e-03,0.001420,1.043715e-07,159,abs,2.097868e-05,4.195735e-06
37,Respiratory rate,Benuron supp 1000 mg,"(0, 6)","(0, 4)",1.101796e-08,0.003749,7.012093e-06,270,abs,1.409431e-03,1.566034e-04
38,Sodium [Moles/volume] in Blood,Benuron supp 1000 mg,"(0, 6)","(0, 4)",9.826408e-06,0.000122,2.478154e-15,157,abs,4.981089e-13,2.490544e-13
39,Calcium.ionized [Moles/volume] in Blood,Dafalgan Brausetbl 1g,"(6, 12)","(0, 4)",1.642939e-02,0.021772,8.501055e-06,121,abs,1.708712e-03,1.708712e-04
74,Metronidazole tabl 200 mg,Dormicum inj 1 mg/ml,"(0, 6)","(0, 4)",2.309590e-03,0.003959,1.909392e-06,229,abs,3.837877e-04,4.818827e-05
190,Calcium.ionized [Moles/volume] in Blood,Zinacef Amp 1.5 g,"(0, 6)","(0, 4)",7.292428e-03,0.042667,1.151588e-06,186,abs,2.314691e-04,3.857819e-05
198,Potassium [Moles/volume] in Blood,Zinacef Amp 1.5 g,"(0, 6)","(0, 4)",1.551256e-03,0.001693,1.917941e-06,190,abs,3.855062e-04,4.818827e-05
200,Sodium [Moles/volume] in Blood,Zinacef Amp 1.5 g,"(0, 6)","(0, 4)",2.183177e-03,0.009609,5.021684e-10,189,abs,1.009359e-07,3.364528e-08


In [ ]:

h_plotter = plots.ClinicalPlotAnalysis(
    data = raw_path,
    res = res_path,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity="", 
    lab_mapping={}
)
h_corrs_data_df = h_plotter.plot(final_h_final_lab_med_data, h_labs, before_windows=before_windows, after_windows=after_windows)